<a href="https://colab.research.google.com/github/ashwinidverma/llm-from-scratch/blob/main/LLM_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import re

text = "Hello, world.  This is a test."
result =  re.split(r'([,.]|\s)',text)
print(result)
result = [item for item in result if item.strip()]
result

['Hello', ',', '', ' ', 'world', '.', '', ' ', '', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


['Hello', ',', 'world', '.', 'This', 'is', 'a', 'test', '.']

In [5]:
with open("/content/the-verdict.txt",'r',encoding='utf-8') as f:
  raw_text = f.read()
print("total no of character:", len(raw_text))


total no of character: 20479


In [6]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [7]:
print(len(preprocessed))

4690


# Token IDs

In [8]:
all_words = sorted(set(preprocessed))
vocab_size= len(all_words)
#
print(vocab_size)

1130


In [9]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [10]:
vocab.items()
for i, item in enumerate(vocab.items()):
  print(item)
  if i >10:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)


In [11]:
class SimpleTokenizerV1:
  def __init__(self,vocab):
    self.str_to_int = vocab
    self.int_to_str = {i:s for s,i in vocab.items()}

  def encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])
      #replace space before the specified punctuations
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text

In [12]:
tokenizer = SimpleTokenizerV1(vocab)
text =""""It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [13]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [14]:
text= "Hello, do you like tea?"
print(tokenizer.encode(text))

KeyError: 'Hello'

Special context tokens: To handal unknow tokens |unk| -> 783 (to assin ids for unknown tokens);
|endoftext| -> 784 (present at the end or begning of the text collected from different sources)

In [15]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}


In [16]:
 len(vocab.items())

1132

In [17]:
for i, item in enumerate(list(vocab.items())[-5:]):
  print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [18]:
class SimpleTokenizerV2:
  def __init__(self,vocab):
    self.str_to_int = vocab
    self.int_tostr = {i:s for s,i in vocab.items()}
  def encode(self,text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [ item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids
  def decode(self,ids):
    text = " ".join([self.int_tostr[i] for i in ids])
    text = re.sub(r'\s+([,.:;?!"()\'])', r'\1',text)
    return text


In [19]:
tokenizer=SimpleTokenizerV2(vocab)
text1= "Hello, do you like tea?"
text2="In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1,text2))
print(text)


Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [20]:
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [21]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

#Byte Pair Encoding (BPE tokenizer)

It is a sub-word tokenization algorithm invented in 1994 for data compression.
"Most common pair of consecutive bytes of data is replaced with a byte that does ot occur in data"

Application in LLM
"BPE ensure that most comman words in the vocabulary are represented as a single token, while rare words are broken down into two or more subword tokens"

stopping criteria: token count or number of
iterations

In [22]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.7 MB/s eta 0:00:00


In [25]:
import importlib
import tiktoken
importlib.metadata.version('tiktoken')

'0.9.0'

In [26]:
tokenizer = tiktoken.get_encoding("gpt2")

In [28]:
text = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces" "of someunknownPlace"
)

integers = tokenizer.encode(text,allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271]


In [29]:
tokenizer.decode(integers)

'Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace'